In [ ]:
import json
import pandas as pd
from pathlib import Path

In [ ]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)

OT_LOGS_DIR = Path("/lustre/groups/eml/projects/sroschmann/ot_logs")

In [ ]:
def parse_params(params_path):
    params = {}
    if not params_path.exists():
        return params

    try:
        with open(params_path, "r") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue

                if ":" in line:
                    key, val = line.split(":", 1)
                elif "=" in line:
                    key, val = line.split("=", 1)
                else:
                    continue

                params[key.strip()] = val.strip()
    except Exception as e:
        print(f"Warning: Could not read {params_path}: {e}")

    return params

In [ ]:
def load_and_flatten_metrics(json_path, prefix):
    metrics = {}
    if not json_path.exists():
        return metrics

    try:
        with open(json_path, "r") as f:
            data = json.load(f)
            epoch_best = data.get("epoch_best", {})

            if isinstance(epoch_best, dict):
                for key, value in epoch_best.items():
                    col_name = f"{prefix}_{key}"
                    metrics[col_name] = value
            else:
                raise ValueError("epoch_best is not a dictionary")

    except Exception as e:
        print(f"Warning: Issue parsing {json_path}: {e}")

    return metrics

In [ ]:
def build_results_dataframe(base_dir):
    data_rows = []
    print(f"Scanning directory: {base_dir} ...")

    # Iterate over all subdirectories in ot_logs
    for exp_dir in base_dir.iterdir():
        if not exp_dir.is_dir():
            continue

        row = {"experiment_id": exp_dir.name, "full_path": str(exp_dir)}

        # Parse Params (Metadata)
        params_path = exp_dir / "params.txt"
        row.update(parse_params(params_path))

        # Load COCO Metrics
        coco_path = exp_dir / "results" / "COCO" / "alignment_probing.json"
        coco_metrics = load_and_flatten_metrics(coco_path, prefix="coco")
        row.update(coco_metrics)

        # Load ImageNet Metrics
        imagenet_path = exp_dir / "results" / "imagenetv1" / "alignment_probing.json"
        imagenet_metrics = load_and_flatten_metrics(imagenet_path, prefix="imagenet")
        row.update(imagenet_metrics)

        data_rows.append(row)

    df = pd.DataFrame(data_rows)
    return df

In [ ]:
def extract_model_and_dataset(path_str):
    if not isinstance(path_str, str):
        return None, None
    
    # Clean artifacts: remove brackets, quotes, commas
    cleaned_str = path_str.replace("[", "").replace("]", "").replace("'", "").replace('"', "").replace(",", " ")
    
    # Split into individual paths
    paths = cleaned_str.split()
    
    models = []
    datasets = []
    
    for p in paths:
        parts = p.strip("/").split("/")
        if len(parts) >= 2:
            models.append(parts[-2]) 
            datasets.append(parts[-1])
            
    if not models:
        return None, None
        
    final_model = models[0]
    final_dataset = " + ".join(datasets)
    
    return final_model, final_dataset

In [ ]:

df = build_results_dataframe(OT_LOGS_DIR)

if df.empty:
    print("No experiments found.")
else:
    # Define base columns (metrics + params)
    param_columns = [
        "seed", "linear-type", "width-factor", "logit_scale", "logit_bias",
        "semisupervised", "n_supervised_pairs", "batch-size-supervised",
        "n_unsupervised_image", "n_unsupervised_text", "anchor_lam_x",
        "anchor_lam_y", "alpha_semisupervised_sail", "alpha_semisupervised_ot",
        "epsilon_sinkhorn_shared", "n_iters_sinkhorn_shared",
        "epsilon_sinkhorn_anchor", "n_iters_sinkhorn_anchor",
        "unsupervised_index_mode",
    ]

    key_metrics = [
        "experiment_id", "coco_T2I R@1", "coco_T2I R@5",
        "coco_I2T R@1", "coco_I2T R@5", "imagenet_top1", "imagenet_top5",
    ]

    # Extract Models and Datasets for ALL embedding columns
    all_embedding_cols = [
        "supervised_image_embedding", "supervised_text_embedding",
        "unsupervised_image_embedding", "unsupervised_text_embedding",
        "val_image_embedding", "val_text_embedding"
    ]

    dataset_cols = []

    for col in all_embedding_cols:
        if col in df.columns:
            base_name = col.replace("_embedding", "")
            extracted = df[col].apply(extract_model_and_dataset)
            
            df[f"{base_name}_model"] = extracted.apply(lambda x: x[0])
            df[f"{base_name}"] = extracted.apply(lambda x: x[1])
            dataset_cols.append(base_name)
            
            df.drop(columns=[col], inplace=True)

    # Assert and merge image models
    if "supervised_image_model" in df.columns and "unsupervised_image_model" in df.columns:
        mismatches = df[df["supervised_image_model"] != df["unsupervised_image_model"]]
        if not mismatches.empty:
            print(f"\n[WARNING] Found {len(mismatches)} experiments where supervised_image_model != unsupervised_image_model")
            print(mismatches[["experiment_id", "supervised_image_model", "unsupervised_image_model"]])
        
        df["image_model"] = df["supervised_image_model"]
        
        cols_to_drop = ["supervised_image_model", "unsupervised_image_model"]
        if "val_image_model" in df.columns:
            cols_to_drop.append("val_image_model")
        df.drop(columns=cols_to_drop, inplace=True)

    # Assert and merge text models
    if "supervised_text_model" in df.columns and "unsupervised_text_model" in df.columns:
        mismatches = df[df["supervised_text_model"] != df["unsupervised_text_model"]]
        if not mismatches.empty:
            print(f"\n[WARNING] Found {len(mismatches)} experiments where supervised_text_model != unsupervised_text_model")
            print(mismatches['experiment_id'])
            print(mismatches[["experiment_id", "supervised_text_model", "unsupervised_text_model"]])

        df["text_model"] = df["supervised_text_model"]
        
        cols_to_drop = ["supervised_text_model", "unsupervised_text_model"]
        if "val_text_model" in df.columns:
            cols_to_drop.append("val_text_model")
        df.drop(columns=cols_to_drop, inplace=True)
    
    dataset_cols.sort()
    model_cols = ["image_model", "text_model"]
    
    final_cols = key_metrics + model_cols + dataset_cols + param_columns
    final_cols = [c for c in final_cols if c in df.columns]

    df = df[final_cols]

    # Sort by main metric
    if "coco_T2I R@1" in df.columns:
        df = df.sort_values(by="coco_T2I R@1", ascending=False)

    print(f"\nLoaded {len(df)} experiments.")

In [ ]:
df